In [1]:
import torch
import torch.nn as nn
import numpy as np
import nettest

SyntaxError: invalid syntax (nettest.py, line 30)

Neural network implementation to model the mathematical statement, $a^2 + b^2 \approx 1$ in the context $a, b \in {\rm I\!R}$ to a predetermined precision in the training set.  

### Network Implementation

In [2]:
class Neural_Network(nn.Module):
    def __init__(self, inputSize, outputSize, hiddenSize, learning_rate):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenSize = hiddenSize
        self.learning_rate = learning_rate
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize).float() # 2 X 3 tensor
        self.W2 = torch.randn(self.hiddenSize, self.outputSize).float() # 3 X 1 tensor
        self.b1 = torch.randn(self.hiddenSize, 1).float().T
        self.b2 = torch.randn(self.outputSize, 1).float().T
        
        
    def forward(self, X):
        #print(X.size(), self.W1.size(), self.b1.size(), self.W2.size())
        self.z = torch.matmul(X, self.W1)
        self.z = self.z + self.b1 # 3 X 3 ".dot" does not broadcast in PyTorch 
        self.z2 = self.sigmoid(self.z) # activation function
        self.z3 = torch.matmul(self.z2, self.W2) + self.b2
        o = self.sigmoid(self.z3) # final activation function
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta) * self.learning_rate
        self.b1 += torch.matmul(torch.from_numpy(np.ones(X.size()[0])).float().T, self.z2_delta)* self.learning_rate
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta) * self.learning_rate
        self.b2 += torch.matmul(torch.from_numpy(np.ones(self.z2.size()[0])).float().T, self.o_delta)* self.learning_rate
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self, inp):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(inp)))

### Acquiring Training Data

In [3]:
X, y = generateData(2000, 0.01)
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

### Training the Network

In [4]:
NN = Neural_Network(2, 1, 3, 0.01)
for i in range(10000):  # trains the NN
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)
#NN.predict()

#0 Loss: 0.28228309750556946
#1 Loss: 0.2634720504283905
#2 Loss: 0.24393896758556366
#3 Loss: 0.22435405850410461
#4 Loss: 0.20494064688682556
#5 Loss: 0.18688037991523743
#6 Loss: 0.17174671590328217
#7 Loss: 0.15938763320446014
#8 Loss: 0.1498885452747345
#9 Loss: 0.1466219127178192
#10 Loss: 0.14531069993972778
#11 Loss: 0.16446048021316528
#12 Loss: 0.14204654097557068
#13 Loss: 0.17284192144870758
#14 Loss: 0.1172289028763771
#15 Loss: 0.13749010860919952
#16 Loss: 0.1131875291466713
#17 Loss: 0.14044080674648285
#18 Loss: 0.09652306139469147
#19 Loss: 0.11458128690719604
#20 Loss: 0.09151933342218399
#21 Loss: 0.11139080673456192
#22 Loss: 0.08187929540872574
#23 Loss: 0.09653470665216446
#24 Loss: 0.07675640285015106
#25 Loss: 0.0904335081577301
#26 Loss: 0.0711958110332489
#27 Loss: 0.08247364312410355
#28 Loss: 0.06706855446100235
#29 Loss: 0.07702918350696564
#30 Loss: 0.063405342400074
#31 Loss: 0.07208700478076935
#32 Loss: 0.06032504513859749
#33 Loss: 0.06802001595497131

#264 Loss: 0.023073257878422737
#265 Loss: 0.02307482436299324
#266 Loss: 0.023029880598187447
#267 Loss: 0.02303173579275608
#268 Loss: 0.02298693172633648
#269 Loss: 0.022989103570580482
#270 Loss: 0.02294440008699894
#271 Loss: 0.022946899756789207
#272 Loss: 0.022902309894561768
#273 Loss: 0.022905118763446808
#274 Loss: 0.022860603407025337
#275 Loss: 0.022863764315843582
#276 Loss: 0.02281930483877659
#277 Loss: 0.022822795435786247
#278 Loss: 0.02277839370071888
#279 Loss: 0.02278224192559719
#280 Loss: 0.02273784950375557
#281 Loss: 0.02274206094443798
#282 Loss: 0.022697681561112404
#283 Loss: 0.02270227111876011
#284 Loss: 0.02265787497162819
#285 Loss: 0.02266284078359604
#286 Loss: 0.02261841669678688
#287 Loss: 0.022623766213655472
#288 Loss: 0.022579306736588478
#289 Loss: 0.022585051134228706
#290 Loss: 0.02254052646458149
#291 Loss: 0.0225466750562191
#292 Loss: 0.02250206284224987
#293 Loss: 0.022508632391691208
#294 Loss: 0.022463927045464516
#295 Loss: 0.022470910102

#523 Loss: 0.019163746386766434
#524 Loss: 0.019064074382185936
#525 Loss: 0.01913772150874138
#526 Loss: 0.01903761550784111
#527 Loss: 0.019111689180135727
#528 Loss: 0.019011151045560837
#529 Loss: 0.019085660576820374
#530 Loss: 0.01898469217121601
#531 Loss: 0.019059622660279274
#532 Loss: 0.01895824447274208
#533 Loss: 0.019033579155802727
#534 Loss: 0.018931781873106956
#535 Loss: 0.019007522612810135
#536 Loss: 0.018905313685536385
#537 Loss: 0.018981462344527245
#538 Loss: 0.01887885108590126
#539 Loss: 0.018955381587147713
#540 Loss: 0.018852388486266136
#541 Loss: 0.01892930269241333
#542 Loss: 0.018825912848114967
#543 Loss: 0.01890319585800171
#544 Loss: 0.018799440935254097
#545 Loss: 0.01887708716094494
#546 Loss: 0.018772965297102928
#547 Loss: 0.018850965425372124
#548 Loss: 0.01874648407101631
#549 Loss: 0.018824851140379906
#550 Loss: 0.018720000982284546
#551 Loss: 0.018798699602484703
#552 Loss: 0.01869349740445614
#553 Loss: 0.018772536888718605
#554 Loss: 0.01866

#781 Loss: 0.015574264340102673
#782 Loss: 0.015501784160733223
#783 Loss: 0.015543852001428604
#784 Loss: 0.015472432598471642
#785 Loss: 0.015513409860432148
#786 Loss: 0.015443047508597374
#787 Loss: 0.015482931397855282
#788 Loss: 0.015413652174174786
#789 Loss: 0.015452424064278603
#790 Loss: 0.015384234488010406
#791 Loss: 0.015421891584992409
#792 Loss: 0.015354801900684834
#793 Loss: 0.015391336753964424
#794 Loss: 0.015325355343520641
#795 Loss: 0.01536075584590435
#796 Loss: 0.015295888297259808
#797 Loss: 0.015330148860812187
#798 Loss: 0.01526640821248293
#799 Loss: 0.015299521386623383
#800 Loss: 0.015236915089190006
#801 Loss: 0.015268874354660511
#802 Loss: 0.015207410790026188
#803 Loss: 0.01523821335285902
#804 Loss: 0.0151778943836689
#805 Loss: 0.01520752813667059
#806 Loss: 0.015148364938795567
#807 Loss: 0.015176836401224136
#808 Loss: 0.015118825249373913
#809 Loss: 0.015146125108003616
#810 Loss: 0.015089282765984535
#811 Loss: 0.015115409158170223
#812 Loss: 0.0

#1037 Loss: 0.012291327118873596
#1038 Loss: 0.01228225976228714
#1039 Loss: 0.012273217551410198
#1040 Loss: 0.012264197692275047
#1041 Loss: 0.01225519459694624
#1042 Loss: 0.012246222235262394
#1043 Loss: 0.012237265706062317
#1044 Loss: 0.012228332459926605
#1045 Loss: 0.01221941877156496
#1046 Loss: 0.012210527434945107
#1047 Loss: 0.012201660312712193
#1048 Loss: 0.012192810885608196
#1049 Loss: 0.012183982878923416
#1050 Loss: 0.012175177223980427
#1051 Loss: 0.012166395783424377
#1052 Loss: 0.012157630175352097
#1053 Loss: 0.012148888781666756
#1054 Loss: 0.012140164151787758
#1055 Loss: 0.012131460942327976
#1056 Loss: 0.012122780084609985
#1057 Loss: 0.012114113196730614
#1058 Loss: 0.012105471454560757
#1059 Loss: 0.012096847407519817
#1060 Loss: 0.01208824384957552
#1061 Loss: 0.01207965798676014
#1062 Loss: 0.012071093544363976
#1063 Loss: 0.012062547728419304
#1064 Loss: 0.01205401960760355
#1065 Loss: 0.012045512907207012
#1066 Loss: 0.012037022970616817
#1067 Loss: 0.01

...

#9179 Loss: 0.003363264724612236
#9180 Loss: 0.003363054245710373
#9181 Loss: 0.003362843068316579
#9182 Loss: 0.0033626353833824396
#9183 Loss: 0.003362425835803151
#9184 Loss: 0.003362217452377081
#9185 Loss: 0.0033620072063058615
#9186 Loss: 0.0033617957960814238
#9187 Loss: 0.0033615881111472845
#9188 Loss: 0.0033613755367696285
#9189 Loss: 0.0033611678518354893
#9190 Loss: 0.003360958304256201
#9191 Loss: 0.0033607473596930504
#9192 Loss: 0.003360541770234704
#9193 Loss: 0.003360331291332841
#9194 Loss: 0.0033601224422454834
#9195 Loss: 0.0033599131274968386
#9196 Loss: 0.003359705675393343
#9197 Loss: 0.0033594954293221235
#9198 Loss: 0.003359285183250904
#9199 Loss: 0.003359076799824834
#9200 Loss: 0.0033588651567697525
#9201 Loss: 0.0033586567733436823
#9202 Loss: 0.0033584453631192446
#9203 Loss: 0.0033582395408302546
#9204 Loss: 0.0033580311574041843
#9205 Loss: 0.0033578197471797466
#9206 Loss: 0.0033576120622456074
#9207 Loss: 0.003357403678819537
#9208 Loss: 0.003357191104

#9423 Loss: 0.0033130149822682142
#9424 Loss: 0.003312811953946948
#9425 Loss: 0.0033126124180853367
#9426 Loss: 0.003312408924102783
#9427 Loss: 0.0033122082240879536
#9428 Loss: 0.0033120051957666874
#9429 Loss: 0.003311802400276065
#9430 Loss: 0.0033116007689386606
#9431 Loss: 0.003311396576464176
#9432 Loss: 0.00331119610927999
#9433 Loss: 0.003310995176434517
#9434 Loss: 0.0033107921481132507
#9435 Loss: 0.0033105919137597084
#9436 Loss: 0.003310390980914235
#9437 Loss: 0.003310190513730049
#9438 Loss: 0.003309988183900714
#9439 Loss: 0.003309784224256873
#9440 Loss: 0.0033095835242420435
#9441 Loss: 0.00330938003025949
#9442 Loss: 0.0033091793302446604
#9443 Loss: 0.0033089795615524054
#9444 Loss: 0.003308778628706932
#9445 Loss: 0.0033085739705711603
#9446 Loss: 0.0033083735033869743
#9447 Loss: 0.00330817187204957
#9448 Loss: 0.003307970007881522
#9449 Loss: 0.0033077700063586235
#9450 Loss: 0.003307567210868001
#9451 Loss: 0.0033073651138693094
#9452 Loss: 0.003307163249701261

#9667 Loss: 0.003264544066041708
#9668 Loss: 0.0032643533777445555
#9669 Loss: 0.0032641561701893806
#9670 Loss: 0.0032639612909406424
#9671 Loss: 0.0032637659460306168
#9672 Loss: 0.003263570833951235
#9673 Loss: 0.003263375721871853
#9674 Loss: 0.0032631810754537582
#9675 Loss: 0.0032629887573421
#9676 Loss: 0.0032627922482788563
#9677 Loss: 0.003262597369030118
#9678 Loss: 0.003262402955442667
#9679 Loss: 0.003262205980718136
#9680 Loss: 0.0032620145939290524
#9681 Loss: 0.003261819714680314
#9682 Loss: 0.0032616236712783575
#9683 Loss: 0.0032614292576909065
#9684 Loss: 0.0032612348441034555
#9685 Loss: 0.003261040197685361
#9686 Loss: 0.0032608439214527607
#9687 Loss: 0.0032606504391878843
#9688 Loss: 0.003260454861447215
#9689 Loss: 0.0032602623105049133
#9690 Loss: 0.0032600644044578075
#9691 Loss: 0.0032598713878542185
#9692 Loss: 0.0032596755772829056
#9693 Loss: 0.0032594804652035236
#9694 Loss: 0.003259287914261222
#9695 Loss: 0.003259092103689909
#9696 Loss: 0.00325890001840

#9911 Loss: 0.0032177367247641087
#9912 Loss: 0.0032175472006201744
#9913 Loss: 0.003217360470443964
#9914 Loss: 0.0032171709463000298
#9915 Loss: 0.0032169839832931757
#9916 Loss: 0.0032167930621653795
#9917 Loss: 0.0032166081946343184
#9918 Loss: 0.0032164191361516714
#9919 Loss: 0.0032162305433303118
#9920 Loss: 0.003216044045984745
#9921 Loss: 0.00321585382334888
#9922 Loss: 0.0032156645320355892
#9923 Loss: 0.003215475706383586
#9924 Loss: 0.003215289209038019
#9925 Loss: 0.003215100383386016
#9926 Loss: 0.0032149115577340126
#9927 Loss: 0.003214724827557802
#9928 Loss: 0.0032145362347364426
#9929 Loss: 0.0032143499702215195
#9930 Loss: 0.003214159980416298
#9931 Loss: 0.003213972318917513
#9932 Loss: 0.003213783260434866
#9933 Loss: 0.0032135946676135063
#9934 Loss: 0.003213408635929227
#9935 Loss: 0.0032132191117852926
#9936 Loss: 0.003213033312931657
#9937 Loss: 0.0032128437887877226
#9938 Loss: 0.0032126540318131447
#9939 Loss: 0.0032124666031450033
#9940 Loss: 0.0032122796401

C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Neural_Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
